In [1]:
%matplotlib qt
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0,'../')
sys.path.insert(0,'../../../data_analysis/')
from simulate_door_slam import *
from trajectory import *
from utils import *
from local_plot import *


In [4]:
path = "/home/xuhao/data/d2slam/quadcam_7inch_n3_2023_1_14/outputs/doorslam-5-yaw"
nodes = [1, 2, 3, 4, 5]
thres = 0.01 #Terminate thresold for DGS
evaluate_door_slam(path, nodes, thres)

paths_pgo, t0 = read_paths(path + "/door-slam/", nodes, prefix="pgo_")
paths_rt, _ = read_paths(path + "/door-slam/", nodes, prefix="realtime_", t0=t0)
paths_gt, _ = read_paths(path + "/../../", nodes, prefix="eight_yaw_", suffix="-groundtruth.txt", t0=t0)
align_paths(paths_rt, paths_gt, align_by_first=True, align_with_minize=True)
align_paths(paths_pgo, paths_gt, align_by_first=True, align_with_minize=True)
dte=550
display(plot_fused_err(nodes, paths_rt, paths_gt, poses_pgo=paths_pgo, dte=dte, show=False))
display(plot_relative_pose_err(1, nodes[1:], paths_rt, paths_gt, dte=dte, show=False))

Data min_iter: 553, max_iter: 591
iter: 44, iterations: 12, max_time: 30.4, total_time 72.1, initial: 6.1, final: 0.0119, volume: 732 (poses)
iter: 45, iterations: 13, max_time: 39.9, total_time 90.6, initial: 12.5, final: 0.0219, volume: 1339 (poses)
iter: 46, iterations: 12, max_time: 36.6, total_time 89.8, initial: 18.9, final: 0.0319, volume: 1668 (poses)
iter: 47, iterations: 11, max_time: 37.4, total_time 86.0, initial: 27.9, final: 0.0451, volume: 1980 (poses)
iter: 48, iterations: 16, max_time: 57.2, total_time 144.8, initial: 34.9, final: 0.0596, volume: 3488 (poses)
iter: 49, iterations: 14, max_time: 54.6, total_time 151.3, initial: 44.2, final: 0.0767, volume: 3696 (poses)
iter: 50, iterations: 16, max_time: 63.4, total_time 195.8, initial: 51.3, final: 0.093, volume: 4928 (poses)
iter: 51, iterations: 14, max_time: 58.8, total_time 175.2, initial: 60.3, final: 0.105, volume: 4914 (poses)
iter: 52, iterations: 13, max_time: 64.4, total_time 164.6, initial: 70.7, final: 0.12

terminate called after throwing an instance of 'gtsam::IndeterminantLinearSystemException'
  what():  
Indeterminant linear system detected while working near variable
7205759403796801416 (Symbol: d4007816).

Thrown when a linear system is ill-posed.  The most common cause for this
error is having underconstrained variables.  Mathematically, the system is
underdetermined.  See the GTSAM Doxygen documentation at
http://borg.cc.gatech.edu/ on gtsam::IndeterminantLinearSystemException for
more information.
Aborted (core dumped)
terminate called after throwing an instance of 'gtsam::IndeterminantLinearSystemException'
  what():  
Indeterminant linear system detected while working near variable
7205759403796802600 (Symbol: d4009000).

Thrown when a linear system is ill-posed.  The most common cause for this
error is having underconstrained variables.  Mathematically, the system is
underdetermined.  See the GTSAM Doxygen documentation at
http://borg.cc.gatech.edu/ on gtsam::IndeterminantLine

Total solve time: 241868.99999999997, average 471.47953216374265 avg iter 14.693957115009747, total volume: 51759886 (poses)


Drone,Traj. Len.,ATE Pos,ATE Att,Cov/m: x,y,z,Cov Att/m,PGO:ATE Pos,ATE Att
1by1,19.3m,0.245,7.198,1.3e-03,1.2e-02,7.6e-03,5.01e-02,0.145,5.342°
2by1,16.0m,0.401,4.998,2.2e-03,7.8e-03,5.2e-04,2.38e-02,0.148,3.471°
3by1,16.7m,0.421,4.271,2.5e-02,5.6e-03,1.7e-02,2.02e-01,0.150,2.872°
4by1,15.4m,0.270,3.273,5.3e-03,1.3e-02,2.0e-02,4.20e-02,0.151,2.843°
5by1,16.1m,0.280,2.996,5.9e-03,1.2e-02,1.7e-02,3.00e-02,0.152,2.839°
Avg.,16.7,0.323,4.55,,,,,0.149,3.47


Relative,EST RMSE: Pos (XYZ),POS,Ang,BIAS: Pos,Ang
1->2,"0.223,0.207,0.062",0.310,2.36°,"-0.091,+0.045,+0.042",+1.90°
1->3,"0.243,0.247,0.114",0.365,1.44°,"-0.065,-0.035,+0.104",+1.10°
1->4,"0.103,0.062,0.031",0.124,1.11°,"-0.000,-0.006,+0.004",+0.66°
1->5,"0.120,0.092,0.043",0.157,1.04°,"0.000,+0.007,+0.021",+0.34°
Avg:,,0.239,1.49°,,


In [4]:
# path = "/home/xuhao/data/d2slam/tum_datasets//outputs/door-slam"
# nodes = [1, 2, 3, 4, 5]
# thres = 0.01 #Terminate thresold for DGS
# evaluate_door_slam(path, nodes, thres)

paths_pgo, t0 = read_paths(path + "/door-slam/", nodes, prefix="pgo_")
paths_rt, _ = read_paths(path + "/door-slam/", nodes, prefix="realtime_", t0=t0)
paths_gt, _ = read_paths(path + "/../../", nodes, prefix="groundtruth_", t0=t0)
align_paths(paths_rt, paths_gt, align_by_first=True, align_with_minize=True)
align_paths(paths_pgo, paths_gt, align_by_first=True, align_with_minize=True)
dte=550
display(plot_fused_err(nodes, paths_rt, paths_gt, poses_pgo=paths_pgo, dte=dte, show=False))
display(plot_relative_pose_err(1, nodes[1:], paths_rt, paths_gt, dte=dte, show=False))

Drone,Traj. Len.,ATE Pos,ATE Att,Cov/m: x,y,z,Cov Att/m,PGO:ATE Pos,ATE Att
1by1,20.4m,0.028,4.865,3.0e-04,1.4e-04,8.8e-04,3.09e-03,0.057,2.319°
2by1,21.0m,0.065,0.634,2.1e-05,1.6e-04,3.0e-03,1.36e-04,0.118,1.677°
3by1,13.4m,0.119,0.943,5.4e-05,3.0e-05,1.3e-04,4.54e-04,0.070,1.341°
4by1,16.8m,0.086,0.638,3.5e-04,3.0e-04,8.8e-04,6.62e-05,0.047,0.875°
5by1,11.4m,0.060,3.844,1.4e-03,6.3e-05,8.3e-04,1.48e-02,0.030,1.151°
Avg.,16.6,0.072,2.19,,,,,0.064,1.47


Relative,EST RMSE: Pos (XYZ),POS,Ang,BIAS: Pos,Ang
1->2,"0.039,0.049,0.065",0.090,2.23°,"-0.008,-0.026,-0.007",+0.43°
1->3,"0.059,0.036,0.110",0.130,2.67°,"-0.049,+0.000,-0.101",+0.17°
1->4,"0.033,0.050,0.074",0.095,2.26°,"0.005,-0.035,-0.048",+0.41°
1->5,"0.044,0.035,0.044",0.072,2.56°,"-0.038,+0.007,-0.025",+0.93°
Avg:,,0.097,2.43°,,
